# Ejemplo de agente local con Ollama para asistente de brochure (PDF de ejemplo)

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
import ollama

## 1. Carga del PDF

In [2]:
loader = PyPDFLoader("Brochure-ejemplo.pdf")
pags = loader.load()

In [3]:
len(pags)

15

In [4]:
print(pags[3].page_content)

Aventura y descanso en un
mismo lugar
Una oportunidad para que tus clientes conozcan 
un paraíso natural incomparable: el Parque 
Nacional Chicaque se ubica a solo 30 minutos de 
Bogotá. Allí podrán vivir una verdadera 
experiencia ecológica con inolvidables 
cabalgatas, espectaculares zonas de camping y 
glamping, increíbles nidos (casas sobre algunos 
árboles) para descansar y obtener  una vista 
incomparable en medio de la naturaleza. 
Los más aventureros tendrán opciones en 
actividades extremas como la tirolesa y el canopy, 
para descender desde las alturas y vivir con 
emoción y adrenalina este fantástico escenario.
Chicaque Natural, mágica y legendaria
Un pueblo mágico estará esperando por tus clientes. 
Ubicado a 53 km de Bogotá: Guatavita tiene un 
increíble estilo colonial y una riqueza histórica que 
le convierten en un lugar imperdible. 
Allí se encuentra una de las lagunas más famosas de 
Colombia, la laguna de Guatavita; reconocida por su 
encanto natural y por evocar el 

## 2. Text Splitter

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        
        chunk_overlap=300,
        length_function=len,
    )

In [6]:
docs = text_splitter.split_documents(pags)

In [7]:
len(docs)

42

## 3. Embeddings

In [8]:
embeddinds = OllamaEmbeddings(model="paraphrase-multilingual:latest")
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddinds, collection_name="brochure-ejemplo")

In [20]:
def ollama_llm(question: str, context: str):
    formatted_prompt = f"Pregunta: {question}\n\nContexto (responde sólo sobre el contexto del texto entregado): {context}"
    
    stream = ollama.chat(
        model='gemma3:4b',
        messages=[{'role': 'user', 'content': formatted_prompt}],
        options={'temperature': 0.5},
        stream=True
    )
    
    full_response = ""
    handle = display(Markdown(""), display_id=True)
    
    for chunk in stream:
        full_response += chunk['message']['content']
        handle.update(Markdown(full_response))
    
    return full_response

In [10]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10, "fetch_k":20, "lambda_mult": 0.5})
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    #print(retrieved_docs)
    formatted_context = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

In [21]:
result= rag_chain("A cuántos kilómetros está Guatavita de Bogotá?")

Según el texto proporcionado, Guatavita está a 53 kilómetros de Bogotá.

In [22]:
result= rag_chain("Cuales son los mejores lugares para hacer cata de Café?")

Basado en el texto proporcionado, los mejores lugares para hacer cata de café en Bogotá son:

*   **Café del Mercado:** Ofrece filtrados y orígenes de café de diferentes zonas de Colombia, con opciones frías y acompañamientos.
*   **Bogotá en Bici:** Permite experiencias sensoriales únicas con café de origen, semi lavados, naturales y orgánicos, incluyendo un laboratorio para aprender sobre el proceso de elaboración.
*   **Arte y Pasión Café - Escuela de Baristas:** Degusta cafés de distintas partes de Colombia, con apoyo a caficultores y opciones para desarrollar habilidades.
*   **Azahar Coﬀee:** Ofrece cafés de pequeños productores colombianos en una zona exclusiva de la ciudad, con una trilogía de cafés de origen y una amplia selección de pastelería.
*   **Café San Alberto:** Un café premiado internacionalmente que ofrece una experiencia intensa con notas afrutadas y dulces, con técnicas innovadoras de filtrado.
*   **Café Catación Pública:** Ofrece una variedad de cafés de origen, semi lavados, naturales, orgánicos y descafeinados, con más de 500 municipios colombianos.
*   **La Perseverancia:** Un lugar donde se puede degustar platos típicos bogotanos y otros de la región y del país, además de conocer la historia y el trabajo de las mujeres en este mercado tradicional.

In [23]:
result= rag_chain("Cuáles museos se recomienda visitar? que sean exclusivamente museos")

Basado en el texto proporcionado, los museos recomendados para visitar en Bogotá son:

*   **Museo del Oro:** Alberga la colección de orfebrería prehispánica más grande del mundo.
*   **Museo Arqueológico, MUSA:** Exhibe artefactos arqueológicos y ofrece una visión de las culturas ancestrales.
*   **Museo Nacional de Colombia:**  Un lugar para conocer las raíces precolombinas y la historia del país.
*   **Museo Botero:** Presenta la colección de obras de Fernando Botero.
*   **Museo Santa Clara:**  Con una colección de pinturas, esculturas y retablos de los siglos XVII y XVIII.
*   **Museo de arte moderno de Bogotá–MAMBO:**  Exhibe arte contemporáneo de artistas nacionales e internacionales.
*   **Museo Internacional de la Esmeralda:**  Permite conocer la historia y la belleza de la esmeralda.
*   **Inmersión muisca:**  Ofrece experiencias relacionadas con la cultura muisca, incluyendo visitas a sitios arqueológicos, degustación de alimentos y actividades tradicionales.

Además, el texto menciona otras atracciones que podrían considerarse museos o experiencias culturales:

*   **Laguna del cacique Guatavita**
*   **Cabildo muisca de Sesquilé**
*   **Villa de Leyva**
*   **Museo Luis Alberto Acuña, Villa de Leyva**
*   **Observatorio solar muisca**
*   **Plaza fundacional e iglesia doctrinera Sáchica**
*   **Parque Nacional Chicaque**
*   **Guatavita**

In [24]:
result= rag_chain("A cuanto tiempo está Chicaque?")

Según el texto, Chicaque es un parque nacional ubicado a 30 minutos de Bogotá. Ofrece cabalgatas, camping, glamping, nidos en los árboles, tirolesa y canopy.

In [25]:
result= rag_chain("Qué altura tiene Monserrate?")

El cerro de Monserrate tiene una altura de cerca de 3.152 metros sobre el nivel del mar.

In [26]:
result= rag_chain("Qué se puede ver en la plza de mercado Paloqeumao?") # Prueba con errores ortográficos

Según el texto, en la Plaza de Mercado Paloquemao se puede ver:

*   Una gran variedad de productos como frutas, lácteos, verduras y flores.
*   Platos típicos colombianos de la capital y de varias regiones del país.
*   Una experiencia que fusiona sabores auténticos con jugos frutales y platos típicos.
*   Una galería de arte (en la Plaza de Mercado La Concordia).

In [27]:
result= rag_chain("Sí sólo tengo 5 días de viaje, cuales son los 5 lugares imperdibles?")

Basado en el texto proporcionado, aquí hay 5 lugares imperdibles para un viaje de 5 días a Bogotá:

1.  **Parque Nacional Chicaque:** Ofrece una combinación de naturaleza, aventura y descanso con cabalgatas, camping, glamping, y actividades extremas.
2.  **Guatavita:** Un pueblo mágico con el estilo colonial y la famosa laguna de Guatavita, reconocida por su importancia cultural y espiritual.
3.  **Monserrate:** Un icónico cerro con un santuario, vistas panorámicas de la ciudad, flora y fauna, y opciones de ascenso a través de senderos, teleférico o funicular.
4.  **Distrito Graﬃti:** Una galería al aire libre con murales de artistas urbanos, ofreciendo una experiencia cultural y visual única.
5.  **Centro Histórico de Bogotá:**  Incluye lugares como el Museo del Oro, el Museo Arqueológico MUSA, la Catedral Primada, la Plaza La Concordia, y la oportunidad de experimentar la gastronomía local y la cultura bogotana.